### 台北市Youbike API，請下載資料，將資料轉成DataFrame

```py
youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
```

In [2]:
import requests
youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res = requests.get(youbike_url)
if res.status_code == 200:
    print('下載成功')
else:
    print('下載失敗')

下載成功


In [3]:
from pydantic import BaseModel, Field, RootModel, field_validator

class Site(BaseModel):
    站名: str = Field(alias='sna')
    區域: str = Field(alias='sarea')
    時間: str = Field(alias='mday')
    地點: str = Field(alias='ar')
    總車輛數: int = Field(alias='total')
    可借數量: int = Field(alias='available_rent_bikes')
    可還數量: int = Field(alias='available_return_bikes')
    經度: float = Field(alias='longitude')
    緯度: float = Field(alias='latitude')
    狀態: bool = Field(alias='act')

    @field_validator('站名')
    @classmethod
    def get_name(cls, value: str) -> str:
        return value.split('_')[-1]

class YouBike(RootModel):
    root: list[Site]
    def __iter__(self):
        return iter(self.root)
    
    def __getitem__(self, item):
        return self.root[item]
    
data: YouBike = YouBike.model_validate_json(res.text)
all_sites: list[dict] = data.model_dump()

In [4]:
import pandas as pd

df1 = pd.DataFrame(all_sites)
df1

,站名,區域,時間,地點,總車輛數,可借數量,可還數量,經度,緯度,狀態
0,捷運科技大樓站,大安區,2024-06-17 14:15:22,復興南路二段235號前,28,3,25,121.54360,25.02605,True
1,復興南路二段273號前,大安區,2024-06-17 14:06:21,復興南路二段273號西側,21,1,20,121.54357,25.02565,True
2,國北教大實小東側門,大安區,2024-06-17 14:13:21,和平東路二段96巷7號,16,15,1,121.54124,25.02429,True
3,和平公園東側,大安區,2024-06-17 14:14:20,和平東路二段118巷33號,11,9,2,121.54282,25.02351,True
4,辛亥復興路口西北側,大安區,2024-06-17 14:13:20,復興南路二段368號,16,12,4,121.54299,25.02153,True
...,...,...,...,...,...,...,...,...,...,...
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 14:08:15,臺大圖書館西南側,30,24,0,121.54031,25.01690,True
1419,臺大黑森林西側,臺大公館校區,2024-06-17 13:58:15,臺大霖澤館南側,20,20,0,121.54347,25.01995,True
1420,臺大獸醫館南側,臺大公館校區,2024-06-17 14:15:22,臺大獸醫系館南側,24,20,1,121.54242,25.01791,True
1421,臺大新體育館東南側,臺大公館校區,2024-06-17 14:10:31,臺大體育館東側,40,20,9,121.53591,25.02112,True
